In [2]:
#可以爬出各大學和超連結版+輸出list+館藏狀態 稍微改過狀態統一的版本

from selenium import webdriver # 先下載 webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
import time

driver = webdriver.Chrome("C:\\Users\\88698\\OneDrive\\文件\\chromedriver") 
driver.get("https://metacat.ntu.edu.tw/") # 更改網址以前往不同網頁

ISBN = input()
element = driver.find_element_by_name('simpleSearchText')
element.send_keys(ISBN)
select = Select(driver.find_element_by_id('simpleType'))
select.select_by_value("ISBN")

# 把不要的勾掉
choose_btn = driver.find_element_by_link_text("機構單位篩選").click()
time.sleep(1)

btn_mid = driver.find_element_by_id('library1').click()
btn_south = driver.find_element_by_id('library2').click()
btn_east = driver.find_element_by_id('library3').click()

no_hsc = driver.find_element_by_id('hsc').click() # 新生醫專
no_tust = driver.find_element_by_id('tust').click() # 大華科大
no_must = driver.find_element_by_id('must').click() # 明新科大
no_taitheo = driver.find_element_by_id('taitheo').click() # 台灣神學研究學院
no_dila = driver.find_element_by_id('dila').click() # 法鼓文理學院
no_yzu = driver.find_element_by_id('yzu').click() # 元智大學
no_niu = driver.find_element_by_id('niu').click() # 宜蘭大學
no_lhu = driver.find_element_by_id('lhu').click() # 龍華科大
no_oit = driver.find_element_by_id('oit').click() # 亞東技術學院
no_ntuvvAlma = driver.find_element_by_id('ntuvvAlma').click() # 原住民圖資中心

save_opt = driver.find_element_by_id("saveOptions").click() # 儲存已選選項
driver.switch_to_alert().accept() # 點選彈出裡面的確定按鈕
close = driver.find_element_by_class_name("close").click() # 按叉叉

search_gogogo = driver.find_element_by_id('simpleSearchButton').click()
time.sleep(70)

# 有"顯示更多"就按下去
more = driver.find_elements_by_name('collapseLink')
for i in range(len(more)):
    more[i].click()
    time.sleep(1)

# 每頁顯示 100 項搜尋結果
try:
    show = Select(driver.find_element_by_name("resultTable_length"))
    show.select_by_value("100")
except:
    show = None
    
name = []
books = driver.find_elements_by_class_name('institution-list')
for i in range(len(books)):
    name.append(books[i].text)

URL = []
www = driver.find_elements_by_class_name('institution-list')
for i in range(len(www)):
    website = www[i].get_attribute('href')
    if website != None:
        URL.append(website)
    else:
        www[i].click()
        www2 = driver.find_elements_by_class_name('institution-list')
        URL.append(www2[-1].get_attribute('href'))
        ActionChains(driver).move_by_offset(150, 200).click().perform()
        ActionChains(driver).move_by_offset(-150, -200).perform()
        
for w in URL:
    web = str(w)
    driver.get(web)
    time.sleep(8)
    status = '不在圖書館內'
    in_lib = ['可外借', '在架上', '在架', '仍在館內', '有可用館藏', '書在館', '目前可獲得', '可獲得', '在書架上', '限館內閱覽']
    
    if 'ntu.primo' in web: #台大系統
        time.sleep(5)
        try:
            back = driver.find_element_by_css_selector(".tab-header .back-button.button-with-icon.zero-margin.md-button.md-primoExplore-theme.md-ink-ripple")
        except:
            back = None
        if back != None:
            back.click()
        thelist = driver.find_elements_by_class_name("md-2-line.md-no-proxy._md")
        for row in thelist:
            plist = row.find_elements_by_tag_name("p")
            where = row.find_elements_by_tag_name("h3")
            i = len(where)
            for sth in plist:
                a = sth.find_elements_by_tag_name("span")
                for _ in range(i):
                    if a[0].text in in_lib:
                        status = '書在館'
                    print('台灣大學', where[_].text, status, a[2].text, w, end = "\n")
                    status = '不在圖書館內'
                    break
                break
            break
        
    elif 'ntnu' in web: #師大系統
        trlist = driver.find_elements_by_class_name('bibItemsEntry')
        for row in trlist:
            tdlist = row.find_elements_by_tag_name('td')
            if tdlist[3].text in in_lib:
                    status = '書在館'
            print('臺灣師範大學', tdlist[0].text, status, w, end = "\n")
            status = '不在圖書館內'
        
    elif 'nccu' in web: #政大系統
        time.sleep(5)
        try:
            back = driver.find_element_by_css_selector(".tab-header .back-button.button-with-icon.zero-margin.md-button.md-primoExplore-theme.md-ink-ripple")
        except:
            back = None
        if back != None:
            back.click()
        thelist = driver.find_elements_by_class_name("md-2-line.md-no-proxy._md")
        for row in thelist:
            plist = row.find_elements_by_tag_name("p")
            where = row.find_elements_by_tag_name("h3")
            i = len(where)
            for sth in plist:
                a = sth.find_elements_by_tag_name("span")
                for _ in range(i):
                    if a[0].text in in_lib:
                        status = '書在館'
                    print('政治大學', where[_].text, status, a[2].text, w, end = "\n")
                    status = '不在圖書館內'
                    break
                break
            break
        
    elif 'ntust' in web: #台科大系統
        trlist = driver.find_elements_by_class_name('bibItemsEntry')
        for row in trlist:
            tdlist = row.find_elements_by_tag_name('td')
            if tdlist[5].text in in_lib:
                    status = '書在館'
            print('臺灣科技大學', tdlist[0].text, status, w, end = "\n")
            status = '不在圖書館內'
        
    elif 'ym' in web: #陽明系統
        trlist = driver.find_elements_by_class_name('bibItemsEntry')
        for row in trlist:
            tdlist = row.find_elements_by_tag_name('td')
            if tdlist[3].text in in_lib:
                status = '書在館'
            print('陽明大學', tdlist[0].text, status, w, end = "\n")
            status = '不在圖書館內'
            
    elif 'ntou' in web: #海大系統
        trlist = driver.find_elements_by_class_name('bibItemsEntry')
        for row in trlist:
            tdlist = row.find_elements_by_tag_name('td')
            if tdlist[3].text in in_lib:
                status = '書在館'
            print('海洋科技大學', tdlist[0].text, status, w, end = "\n")
            status = '不在圖書館內'
    
    elif 'nctu' in web: #交大系統
        time.sleep(5)
        try:
            back = driver.find_element_by_css_selector(".tab-header .back-button.button-with-icon.zero-margin.md-button.md-primoExplore-theme.md-ink-ripple")
        except:
            back = None
        if back != None:
            back.click()
        thelist = driver.find_elements_by_class_name("tab-content-header.margin-bottom-small.margin-left-medium.layout-align-space-between-end.layout-row")
        for row in thelist:
            plist = row.find_elements_by_tag_name("p")
            where = row.find_elements_by_tag_name("h4")
            i = len(where)
            for sth in plist:
                a = sth.find_elements_by_tag_name("span")
                for _ in range(i):
                    if a[0].text in in_lib:
                        status = '書在館'
                    print('交通大學', where[_].text, status, a[2].text, w, end = "\n")
                    status = '不在圖書館內'
                    break
                break
            break
        
    elif 'nthu' in web: #清華系統
        where = driver.find_elements_by_link_text('全部館藏')[1].click()
        table = driver.find_element_by_xpath("/html/body/table[12]")
        trlist = table.find_elements_by_tag_name('tr')
        for row in trlist:
            tdlist = row.find_elements_by_tag_name('td')
            for sth in tdlist:
                if tdlist[8].text in in_lib:
                    status = '書在館'
                print('清華大學', tdlist[2].text, status, w, end = "\n")
                status = '不在圖書館內'
                break
        
    elif 'ncu' in web: #中央系統
        trlist = driver.find_elements_by_class_name('bibItemsEntry')
        for row in trlist:
            tdlist = row.find_elements_by_tag_name('td')
            if tdlist[3].text in in_lib:
                status = '書在館'
            print('中央大學', tdlist[0].text, status, w, end = "\n")
            status = '不在圖書館內'
            
    elif 'ncl' in web: #國圖系統
        where = driver.find_element_by_link_text("書在哪裡(請點選)").click()
        time.sleep(2)
        tdlist = driver.find_elements_by_class_name('td1')
        if tdlist[6].text in in_lib:
                status = '書在館'
        print('國家圖書館', tdlist[1].text, status, w, end = "\n")
        status = '不在圖書館內'
        
    elif 'las.sinica' in web: #中研院系統
        trlist = driver.find_elements_by_class_name('bibItemsEntry')
        for row in trlist:
            tdlist = row.find_elements_by_tag_name('td')
            if tdlist[2].text in in_lib:
                status = '書在館'
            print('中央研究院',tdlist[0].text, status, w, end = "\n")
            status = '不在圖書館內'
            
    elif 'ntpu' in web: #北大系統
        table = driver.find_element_by_class_name("book_location")
        trlist = table.find_elements_by_class_name('odd')
        for row in trlist:
            tdlist = row.find_elements_by_tag_name('td')
            for sth in tdlist:
                if tdlist[2].text in in_lib:
                    status = '書在館'
                print('台北大學', status, w, end = "\n")
                status = '不在圖書館內'
                break
                
    elif '203.64.5.158' in web: #台北藝術大學系統
        where = driver.find_element_by_xpath("/html/body/div/div[1]/div[2]/div/div/div[2]/div[3]/div[1]/div[3]/div/ul/li/div/div[2]/h3/a").click()
        time.sleep(2)
        table = driver.find_element_by_class_name('table.table-bordered')
        trlist = table.find_elements_by_tag_name('tr')
        for row in trlist:
            tdlist = row.find_elements_by_tag_name('td')
            for sth in tdlist:
                if tdlist[5].text in in_lib:
                    status = '書在館'
                print('台北藝術大學', tdlist[0].text, status, w, end = "\n")
                status = '不在圖書館內'
                break
                
    elif '140.131.94.8' in web: #護理健康大學系統(呈現的是書的所在地)
        table = driver.find_element_by_xpath("/html/body/div[4]/div[1]/form[1]/div[2]/div[1]/ul[3]/li[1]/ul[3]/li[2]/table")
        trlist = table.find_elements_by_tag_name('tr')
        for _ in trlist:
            tdlist = driver.find_elements_by_class_name('holdingslist')
            if ('到期日' not in tdlist[3].text) or ('訂購中' not in tdlist[3].text):
                status = '書在館'
            print('台北護理健康大學', status, w, end = "\n")
            status = '不在圖書館內'
            break
            
    elif 'ntut' in web: #北科系統
        table = driver.find_element_by_class_name('order')
        trlist = table.find_elements_by_tag_name('tr')
        for row in trlist:
            tdlist = row.find_elements_by_tag_name('td')
            for sth in tdlist:
                if tdlist[8].text in in_lib:
                    status = '書在館'
                print('台北科技大學', tdlist[2].text, status, w, end = "\n")
                status = '不在圖書館內'
                break    
                
    elif 'ntl' in web: #國台圖系統
        where = driver.find_element_by_xpath("/html/body/table[8]/tbody/tr[1]/td[2]/a").click()
        time.sleep(2)
        tdlist = driver.find_elements_by_class_name('td1')
        if tdlist[14].text in in_lib:
                status = '書在館'
        print('國立台灣圖書館', tdlist[7].text, tdlist[8].text, status, w, end = "\n") 
        status = '不在圖書館內'
        
    elif 'itrilib' in web: #工研院系統
        where = driver.find_element_by_xpath("/html/body/table[8]/tbody/tr[1]/td[2]/a").click()
        time.sleep(2)
        tdlist = driver.find_elements_by_class_name('td1')
        if tdlist[12].text in in_lib:
            status = '書在館'
        print('工業研究院', tdlist[9].text, tdlist[10].text, status, w, end = "\n")
        status = '不在圖書館內'
        
    elif 'nhri' in web: #國衛院系統
        where = driver.find_element_by_xpath('/html/body/table[8]/tbody/tr[1]/td[2]/a').click()
        time.sleep(2)
        tdlist = driver.find_elements_by_class_name('td1')
        if tdlist[13].text in in_lib:
            status = '書在館'
        print('國家衛生研究院', tdlist[6].text, tdlist[7].text, status, w, end = "\n")
        status = '不在圖書館內'
        
    elif 'webpac.pccu' in web: #中國文化大學系統
        trlist = driver.find_elements_by_class_name('bibItemsEntry')
        for row in trlist:
            tdlist = row.find_elements_by_tag_name('td')
            if tdlist[3].text in in_lib:
                    status = '書在館'
            print('中國文化大學', tdlist[0].text, status, w, end = "\n")
            status = '不在圖書館內'

    elif 'shu' in web: #世新系統       
        where = driver.find_element_by_link_text("世新圖書館").click()
        table = driver.find_element_by_xpath("/html/body/p[2]/table[1]")
        trlist = table.find_elements_by_tag_name('tr')
        for row in trlist:
            tdlist = row.find_elements_by_tag_name('td')
            for sth in tdlist:
                if tdlist[3].text in in_lib:
                    status = '書在館'
                print('世新大學', tdlist[1].text, status, w, end = "\n")
                status = '不在圖書館內'
                break
                
    elif 'uco-network' in web: #淡江系統(觀察中)
        ime.sleep(5)
        try:
            back = driver.find_element_by_css_selector(".tab-header .back-button.button-with-icon.zero-margin.md-button.md-primoExplore-theme.md-ink-ripple")
        except:
            back = None
        if back != None:
            back.click()
        thelist = driver.find_elements_by_class_name("md-2-line.md-no-proxy._md")
        for row in thelist:
            plist = row.find_elements_by_tag_name("p")
            where = row.find_elements_by_tag_name("h3")
            i = len(where)
            for sth in plist:
                a = sth.find_elements_by_tag_name("span")
                for _ in range(i):
                    if a[0].text in in_lib:
                        status = '書在館'
                    print('淡江大學', where[_].text, status, a[2].text, w, end = "\n")
                    status = '不在圖書館內'
                    break
                break
            break
        
    elif 'fju' in web: #輔仁大學系統
        trlist = driver.find_elements_by_class_name('bibItemsEntry')
        for row in trlist:
            tdlist = row.find_elements_by_tag_name('td')
            if tdlist[3].text in in_lib:
                    status = '書在館'
            print('輔仁大學', tdlist[0].text, status, w, end = "\n")
            status = '不在圖書館內'

    elif 'mcu' in web: #銘傳系統
        time.sleep(5)
        try:
            back = driver.find_element_by_css_selector(".tab-header .back-button.button-with-icon.zero-margin.md-button.md-primoExplore-theme.md-ink-ripple")
        except:
            back = None
        if back != None:
            back.click()
        thelist = driver.find_elements_by_class_name("md-2-line.md-no-proxy._md")
        for row in thelist[:-2]:
            plist = row.find_elements_by_tag_name("p")
            where = row.find_elements_by_tag_name("h3")
            i = len(where)
            for sth in plist:
                a = sth.find_elements_by_tag_name("span")
                for _ in range(1):
                    if a[0].text in in_lib:
                        status = '書在館'
                    print('銘傳大學', where[_].text, status, a[2].text, w, end = "\n")
                    status = '不在圖書館內'
                    break
                break
            break
        
    elif 'scu' in web: #東吳系統
        time.sleep(5)
        try:
            back = driver.find_element_by_css_selector(".tab-header .back-button.button-with-icon.zero-margin.md-button.md-primoExplore-theme.md-ink-ripple")
        except:
            back = None
        if back != None:
            back.click()
        thelist = driver.find_elements_by_class_name("md-2-line.md-no-proxy._md")
        for row in thelist[:-2]:
            plist = row.find_elements_by_tag_name("p")
            where = row.find_elements_by_tag_name("h3")
            i = len(where)
            for sth in plist:
                a = sth.find_elements_by_tag_name("span")
                for _ in range(1):
                    if a[0].text in in_lib:
                        status = '書在館'
                    print('東吳大學', where[_].text, status, a[2].text, w, end = "\n")
                    status = '不在圖書館內'
                    break
                break
            break
        
    elif 'cylis' in web: #中原大學系統
        trlist = driver.find_elements_by_class_name('bibItemsEntry')
        for row in trlist:
            tdlist = row.find_elements_by_tag_name('td')
            if tdlist[3].text in in_lib:
                status = '書在館'
            print('中原大學', tdlist[0].text, status, w, end = "\n")
            status = '不在圖書館內'
        
    elif 'tmu' in web: #北醫系統
        table = driver.find_element_by_class_name('order')
        trlist = table.find_elements_by_tag_name('tr')
        for row in trlist:
            tdlist = row.find_elements_by_tag_name('td')
            for sth in tdlist:
                if tdlist[5].text in in_lib:
                    status = '書在館'
                print('台北醫學大學', tdlist[2].text, status, w, end = "\n")
                status = '不在圖書館內'
                break
    
    elif 'moc' in web: #台博館系統(待處理)
        where = driver.find_element_by_link_text("書在哪裡 ? (請點我)").click()
        table = driver.find_element_by_xpath("/html/body/table[9]")
        trlist = table.find_elements_by_tag_name('tr')
        for row in trlist:
            tdlist = row.find_elements_by_tag_name('td')
            for sth in tdlist:
                if tdlist[7].text in in_lib:
                    status = '書在館'
                print('國立台灣博物館', tdlist[2].text, tdlist[3].text, status, w, end = "\n")
                status = '不在圖書館內'
                break
        
    elif 'jinwen' in web: #景文科大系統
        table = driver.find_element_by_class_name('order')
        trlist = table.find_elements_by_tag_name('tr')
        for row in trlist:
            tdlist = row.find_elements_by_tag_name('td')
            for sth in tdlist:
                if tdlist[2].text in in_lib:
                    status = '書在館'
                print('景文科技大學', tdlist[2].text, status, w, end = "\n")
                status = '不在圖書館內'
                break
    
    elif 'mmc' in web: #馬偕醫護系統
        where = driver.find_element_by_xpath("/html/body/table[8]/tbody/tr[1]/td[2]/span/a[1]").click()
        time.sleep(2)
        tdlist = driver.find_elements_by_class_name('td1')
        if tdlist[12].text in in_lib:
            status = '書在館'
        print('馬偕醫護管理專科學校', tdlist[7].text, status, w, end = "\n")
        status = '不在圖書館內'
          
    elif 'tfi' in web: #視聽文化中心系統
        table = driver.find_element_by_xpath("/html/body/div/div[1]/div/div/div/div[2]/div/div[2]/div[3]/div/div[4]")
        trlist = table.find_elements_by_tag_name('tr')
        for row in trlist:
            tdlist = row.find_elements_by_tag_name('td')
            for sth in tdlist:
                if tdlist[4].text in in_lib:
                    status = '書在館'
                print('國家視聽文化中心', tdlist[1].text, status, w, end = "\n")
                status = '不在圖書館內'
                break   

driver.close()

9570825685


<ipython-input-2-ae4d7f5c9d5b>:38: DeprecationWarning: use driver.switch_to.alert instead
  driver.switch_to_alert().accept() # 點選彈出裡面的確定按鈕


台灣大學 圖資系圖書室 書在館 圖資系圖兒童讀物區 https://ntu.primo.exlibrisgroup.com/discovery/fulldisplay?context=L&vid=886NTU_INST:886NTU_INST&lang=zh-tw&docid=alma991025786309704786
中央大學  總圖2F中文書庫 不在圖書館內 http://opac.lib.ncu.edu.tw/search*cht/?searchtype=i&searcharg=9570825685
工業研究院 綜研館 中興51館B2_福委會圖書區 不在圖書館內 http://itrilib.itri.org.tw/F?func=direct&l_base=ITRIBIB&format=999&doc_number=002271101
交通大學 交大圖書總館 書在館 中文圖書區(四樓) http://ustcate.lib.nctu.edu.tw/NCTU:NCTU_ALEPH000902027
台北藝術大學 總圖 書在館 http://203.64.5.158/webpac/search/?field=ref_id&op=AND&type=&q=005040017
東吳大學 雙溪中正書庫 書在館 雙溪中正書庫 https://uco-scu.primo.exlibrisgroup.com/discovery/fulldisplay?context=L&vid=886UCO_SCU:886SCU_INST&lang=zh-tw&docid=alma991002418899705913
政治大學 政大實小 書在館 政大實小圖書室 https://nccu.primo.exlibrisgroup.com/discovery/fulldisplay?context=L&vid=886NCCU_INST:886NCCU_INST&lang=zh-tw&docid=alma991020796779705721
銘傳大學 桃園館 書在館 中文書庫 https://uco-mcu.primo.exlibrisgroup.com/discovery/fulldisplay?context=L&vid=886UCO_MCU:886MCU_INST&lang=zh-tw&doc

In [ ]:
from selenium import webdriver 

import time

driver = webdriver.Chrome("C:\\Users\\88698\\OneDrive\\文件\\chromedriver") 
web = input()
driver.get(web)
if ('primo' or 'ntcu') in web: #台大+政大+銘傳+東吳系統
    time.sleep(5)
    thelist = driver.find_element_by_class_name("separate-list-items.margin-bottom-medium.padding-bottom-zero.md-primoExplore-theme")
    print(thelist.text)
        
elif 'ntnu' in web: #師大系統
    trlist = driver.find_elements_by_class_name('bibItemsEntry')
    for row in trlist:
        tdlist = row.find_elements_by_tag_name('td')
        print('臺灣師範大學', tdlist[0].text, tdlist[3].text, end = "\n")
        

elif 'ntust' in web: #台科大系統
    trlist = driver.find_elements_by_class_name('bibItemsEntry')
    for row in trlist:
        tdlist = row.find_elements_by_tag_name('td')
        print('臺灣科技大學', tdlist[0].text, tdlist[5].text, end = "\n")
        
elif 'ym' in web: #陽明系統
    trlist = driver.find_elements_by_class_name('bibItemsEntry')
    for row in trlist:
        tdlist = row.find_elements_by_tag_name('td')
        print('陽明大學', tdlist[0].text, tdlist[3].text, end = "\n")
            
elif 'ntou' in web: #海大系統
    trlist = driver.find_elements_by_class_name('bibItemsEntry')
    for row in trlist:
        tdlist = row.find_elements_by_tag_name('td')
        print('海洋科技大學', tdlist[0].text, tdlist[3].text, end = "\n")
            

elif 'nthu' in web: #清華系統
    where = driver.find_elements_by_link_text('全部館藏')[1].click()
    table = driver.find_element_by_xpath("/html/body/table[12]")
    trlist = table.find_elements_by_tag_name('tr')
    for row in trlist:
        tdlist = row.find_elements_by_tag_name('td')
        for sth in tdlist:
            print('清華大學', tdlist[2].text, tdlist[8].text ,end = "\n")
            break
        
elif 'ncu' in web: #中央系統
    trlist = driver.find_elements_by_class_name('bibItemsEntry')
    for row in trlist:
        tdlist = row.find_elements_by_tag_name('td')
        print('中央大學', tdlist[0].text, tdlist[3].text, end = "\n")
            
elif 'ncl' in web: #國圖系統
    where = driver.find_element_by_link_text("書在哪裡(請點選)").click()
    time.sleep(2)
    tdlist = driver.find_elements_by_class_name('td1')
    print('國家圖書館', tdlist[1].text, tdlist[6].text, end = "\n")  
        
elif 'las.sinica' in web: #中研院系統
    trlist = driver.find_elements_by_class_name('bibItemsEntry')
    for row in trlist:
        tdlist = row.find_elements_by_tag_name('td')
        print('中央研究院',tdlist[0].text, tdlist[2].text, end = "\n")
            
elif 'ntpu' in web: #北大系統
    table = driver.find_element_by_class_name("book_location")
    trlist = table.find_elements_by_class_name('odd')
    for row in trlist:
        tdlist = row.find_elements_by_tag_name('td')
        for sth in tdlist:
            print('台北大學', tdlist[2].text ,end = "\n")
            break
                
elif '203.64.5.158' in web: #台北藝術大學系統
    where = driver.find_element_by_xpath("/html/body/div/div[1]/div[2]/div/div/div[2]/div[3]/div[1]/div[3]/div/ul/li/div/div[2]/h3/a").click()
    time.sleep(2)
    table = driver.find_element_by_class_name('table.table-bordered')
    trlist = table.find_elements_by_tag_name('tr')
    for row in trlist:
        tdlist = row.find_elements_by_tag_name('td')
        for sth in tdlist:
            print('台北藝術大學', tdlist[0].text, tdlist[5].text, end = "\n")
            break
                
elif '140.131.94.8' in web: #護理健康大學系統
    table = driver.find_element_by_xpath("/html/body/div[4]/div[1]/form[1]/div[2]/div[1]/ul[3]/li[1]/ul[3]/li[2]/table")
    trlist = table.find_elements_by_tag_name('tr')
    for _ in trlist:
        tdlist = driver.find_elements_by_class_name('holdingslist')
        print('台北護理健康大學', tdlist[3].text)
        break
            
elif 'ntut' in web: #北科系統
    table = driver.find_element_by_class_name('order')
    trlist = table.find_elements_by_tag_name('tr')
    for row in trlist:
        tdlist = row.find_elements_by_tag_name('td')
        for sth in tdlist:
            print('台北科技大學', tdlist[2].text, tdlist[8].text, end = "\n")
            break    
                
elif 'ntl' in web: #國台圖系統
    where = driver.find_element_by_xpath("/html/body/table[8]/tbody/tr[1]/td[2]/a").click()
    time.sleep(2)
    tdlist = driver.find_elements_by_class_name('td1')
    print('國立台灣圖書館', tdlist[8].text, tdlist[9].text, tdlist[15].text, end = "\n") 
        
elif 'itrilib' in web: #工研院系統
    where = driver.find_element_by_xpath("/html/body/table[8]/tbody/tr[1]/td[2]/a").click()
    time.sleep(2)
    tdlist = driver.find_elements_by_class_name('td1')
    print('工業研究院', tdlist[9].text, tdlist[10].text, tdlist[12].text, end = "\n")
        
elif 'nhri' in web: #國衛院系統
    where = driver.find_element_by_xpath('/html/body/table[8]/tbody/tr[1]/td[2]/a').click()
    time.sleep(2)
    tdlist = driver.find_elements_by_class_name('td1')
    print('國家衛生研究院', tdlist[6].text, tdlist[7].text, tdlist[13].text, end = "\n")
        
elif 'pccu' in web: #中國文化大學系統
    trlist = driver.find_elements_by_class_name('bibItemsEntry')
    for row in trlist:
        tdlist = row.find_elements_by_tag_name('td')
        print('中國文化大學', tdlist[0].text, tdlist[3].text, end = "\n")

elif 'shu' in web: #世新系統       
    where = driver.find_element_by_link_text("世新圖書館").click()
    table = driver.find_element_by_xpath("/html/body/p[2]/table[1]")
    trlist = table.find_elements_by_tag_name('tr')
    for row in trlist:
        tdlist = row.find_elements_by_tag_name('td')
        for sth in tdlist:
            print('世新大學', tdlist[1].text, tdlist[3].text ,end = "\n")
            break
                

elif 'fju' in web: #輔仁大學系統
    trlist = driver.find_elements_by_class_name('bibItemsEntry')
    for row in trlist:
        tdlist = row.find_elements_by_tag_name('td')
        print('輔仁大學', tdlist[0].text, tdlist[3].text, end = "\n")


elif 'cylis' in web: #中原大學系統
    trlist = driver.find_elements_by_class_name('bibItemsEntry')
    for row in trlist:
        tdlist = row.find_elements_by_tag_name('td')
        print('中原大學', tdlist[0].text, tdlist[3].text, end = "\n")
        
elif 'tmu' in web: #北醫系統
    table = driver.find_element_by_class_name('order')
    trlist = table.find_elements_by_tag_name('tr')
    for row in trlist:
        tdlist = row.find_elements_by_tag_name('td')
        for sth in tdlist:
            print('台北醫學大學', tdlist[2].text, tdlist[5].text, end = "\n")
            break
    
elif 'moc' in web: #台博館系統
    where = driver.find_element_by_link_text("書在哪裡 ? (請點我)").click()
    table = driver.find_element_by_xpath("/html/body/table[9]")
    trlist = table.find_elements_by_tag_name('tr')
    for row in trlist:
        tdlist = row.find_elements_by_tag_name('td')
        for sth in tdlist:
            print('國立台灣博物館', tdlist[2].text, tdlist[3].text, tdlist[7].text ,end = "\n")
            break
        
elif 'jinwen' in web: #景文科大系統
    table = driver.find_element_by_class_name('order')
    trlist = table.find_elements_by_tag_name('tr')
    for row in trlist:
        tdlist = row.find_elements_by_tag_name('td')
        for sth in tdlist:
            print('景文科技大學', tdlist[2].text, tdlist[5].text, end = "\n")
            break
    
elif 'mmc' in web: #馬偕醫護系統
    where = driver.find_element_by_xpath("/html/body/table[8]/tbody/tr[1]/td[2]/span/a[1]").click()
    time.sleep(2)
    tdlist = driver.find_elements_by_class_name('td1')
    print('馬偕醫護管理專科學校', tdlist[7].text, tdlist[12].text, end = "\n")
          
elif 'tfi' in web: #視聽文化中心系統
    table = driver.find_element_by_xpath("/html/body/div/div[1]/div/div/div/div[2]/div/div[2]/div[3]/div/div[4]")
    trlist = table.find_elements_by_tag_name('tr')
    for row in trlist:
        tdlist = row.find_elements_by_tag_name('td')
        for sth in tdlist:
            print('國家視聽文化中心', tdlist[1].text, tdlist[4].text ,end = "\n")
            break
        


In [13]:
from selenium import webdriver
import time
driver = webdriver.Chrome("C:\\Users\\88698\\OneDrive\\文件\\chromedriver") 

driver.get('https://ntu.primo.exlibrisgroup.com/discovery/fulldisplay?docid=alma991024776839704786&context=L&vid=886NTU_INST:886NTU_INST&lang=zh-tw&search_scope=MyInst_and_CI&adaptor=Local%20Search%20Engine&tab=Everything&query=any,contains,%E6%96%B0%E7%AB%B6%E7%88%AD%E6%99%82%E4%BB%A3&offset=0')

time.sleep(8)
try:
    back = driver.find_element_by_css_selector(".tab-header .back-button.button-with-icon.zero-margin.md-button.md-primoExplore-theme.md-ink-ripple")
except:
    back = None
if back != None:
    back.click()
thelist = driver.find_elements_by_class_name("md-2-line.md-no-proxy._md")
for row in thelist:
    plist = row.find_elements_by_tag_name("p")
    where = row.find_elements_by_tag_name("h3")
    i = len(where)
    for sth in plist:
        a = sth.find_elements_by_tag_name("span")
        for _ in range(i): 
            print('台灣大學', where[_].text, a[0].text, end = "\n")
            break
        break
    

台灣大學 總圖書館 有可用館藏
台灣大學  
台灣大學  


In [12]:
#可以爬出各大學和超連結版+輸出list+館藏狀態 稍微改過狀態統一的版本2021.06.07

from selenium import webdriver # 先下載 webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
import time

driver = webdriver.Chrome("C:\\Users\\88698\\OneDrive\\文件\\chromedriver") #記得改位置
driver.get("https://metacat.ntu.edu.tw/") # 更改網址以前往不同網頁

#改成用ISBN查
ISBN = input()
element = driver.find_element_by_name('simpleSearchText')
element.send_keys(ISBN)
select = Select(driver.find_element_by_id('simpleType'))
select.select_by_value("ISBN")

# 把不要的勾掉
choose_btn = driver.find_element_by_link_text("機構單位篩選").click()
time.sleep(1)

btn_mid = driver.find_element_by_id('library1').click()
btn_south = driver.find_element_by_id('library2').click()
btn_east = driver.find_element_by_id('library3').click()

no_hsc = driver.find_element_by_id('hsc').click() # 新生醫專
no_tust = driver.find_element_by_id('tust').click() # 大華科大
no_must = driver.find_element_by_id('must').click() # 明新科大
no_taitheo = driver.find_element_by_id('taitheo').click() # 台灣神學研究學院
no_dila = driver.find_element_by_id('dila').click() # 法鼓文理學院
no_yzu = driver.find_element_by_id('yzu').click() # 元智大學
no_niu = driver.find_element_by_id('niu').click() # 宜蘭大學
no_lhu = driver.find_element_by_id('lhu').click() # 龍華科大
no_oit = driver.find_element_by_id('oit').click() # 亞東技術學院
no_ntuvvAlma = driver.find_element_by_id('ntuvvAlma').click() # 原住民圖資中心

save_opt = driver.find_element_by_id("saveOptions").click() # 儲存已選選項
driver.switch_to_alert().accept() # 點選彈出裡面的確定按鈕
close = driver.find_element_by_class_name("close").click() # 按叉叉

search_gogogo = driver.find_element_by_id('simpleSearchButton').click()
time.sleep(70)

# 有"顯示更多"就按下去
more = driver.find_elements_by_name('collapseLink')
for i in range(len(more)):
    more[i].click()
    time.sleep(1)

# 每頁顯示 100 項搜尋結果
try:
    show = Select(driver.find_element_by_name("resultTable_length"))
    show.select_by_value("100")
except:
    show = None
    
#爬 Metacat
if show != None:
    name = []
    books = driver.find_elements_by_class_name('institution-list')
    for i in range(len(books)):
        name.append(books[i].text)

    URL = []
    www = driver.find_elements_by_class_name('institution-list')
    for i in range(len(www)):
        website = www[i].get_attribute('href')
        if website != None:
            URL.append(website)
        else:
            www[i].click()
            www2 = driver.find_elements_by_class_name('institution-list')
            URL.append(www2[-1].get_attribute('href'))
            ActionChains(driver).move_by_offset(150, 200).click().perform()
            ActionChains(driver).move_by_offset(-150, -200).perform()

    #爬各大學網站
    for w in URL:
        web = str(w)
        driver.get(web)
        time.sleep(8)
        status = '不在圖書館內'
        in_lib = ['可外借', '在架上', '在架', '仍在館內', '有可用館藏', '書在館', '目前可獲得', '可獲得', '在書架上', '限館內閱覽', '在館內']

        if 'ntu.primo' in web: #台大系統
            time.sleep(5)
            try:
                back = driver.find_element_by_css_selector(".tab-header .back-button.button-with-icon.zero-margin.md-button.md-primoExplore-theme.md-ink-ripple")
            except:
                back = None
            if back != None:
                back.click()
            thelist = driver.find_elements_by_class_name("md-2-line.md-no-proxy._md")
            for row in thelist:
                plist = row.find_elements_by_tag_name("p")
                where = row.find_elements_by_tag_name("h3")
                i = len(where)
                for sth in plist:
                    a = sth.find_elements_by_tag_name("span")
                    for _ in range(i):
                        if a[0].text in in_lib:
                            status = '書在館'
                        print('台灣大學', where[_].text, status, w, end = "\n")
                        status = '不在圖書館內'
                        break
                    break
                

        elif 'ntnu' in web: #師大系統
            trlist = driver.find_elements_by_class_name('bibItemsEntry')
            for row in trlist:
                tdlist = row.find_elements_by_tag_name('td')
                if tdlist[3].text in in_lib:
                        status = '書在館'
                print('臺灣師範大學', tdlist[0].text, status, w, end = "\n")
                status = '不在圖書館內'

        elif 'nccu' in web: #政大系統
            time.sleep(5)
            try:
                back = driver.find_element_by_css_selector(".tab-header .back-button.button-with-icon.zero-margin.md-button.md-primoExplore-theme.md-ink-ripple")
            except:
                back = None
            if back != None:
                back.click()
            thelist = driver.find_elements_by_class_name("md-2-line.md-no-proxy._md")
            for row in thelist:
                plist = row.find_elements_by_tag_name("p")
                where = row.find_elements_by_tag_name("h3")
                i = len(where)
                for sth in plist:
                    a = sth.find_elements_by_tag_name("span")
                    for _ in range(i):
                        if a[0].text in in_lib:
                            status = '書在館'
                        print('政治大學', where[_].text, status, w, end = "\n")
                        status = '不在圖書館內'
                        break
                    break
            

        elif 'ntust' in web: #台科大系統
            trlist = driver.find_elements_by_class_name('bibItemsEntry')
            for row in trlist:
                tdlist = row.find_elements_by_tag_name('td')
                if tdlist[5].text in in_lib:
                        status = '書在館'
                print('臺灣科技大學', tdlist[0].text, status, w, end = "\n")
                status = '不在圖書館內'

        elif 'ym' in web: #陽明系統
            trlist = driver.find_elements_by_class_name('bibItemsEntry')
            for row in trlist:
                tdlist = row.find_elements_by_tag_name('td')
                if tdlist[3].text in in_lib:
                    status = '書在館'
                print('陽明大學', tdlist[0].text, status, w, end = "\n")
                status = '不在圖書館內'

        elif 'ntou' in web: #海大系統
            trlist = driver.find_elements_by_class_name('bibItemsEntry')
            for row in trlist:
                tdlist = row.find_elements_by_tag_name('td')
                if tdlist[3].text in in_lib:
                    status = '書在館'
                print('海洋科技大學', tdlist[0].text, status, w, end = "\n")
                status = '不在圖書館內'

        elif 'nctu' in web: #交大系統
            time.sleep(5)
            try:
                back = driver.find_element_by_css_selector(".tab-header .back-button.button-with-icon.zero-margin.md-button.md-primoExplore-theme.md-ink-ripple")
            except:
                back = None
            if back != None:
                back.click()
            thelist = driver.find_elements_by_class_name("tab-content-header.margin-bottom-small.margin-left-medium.layout-align-space-between-end.layout-row")
            for row in thelist:
                plist = row.find_elements_by_tag_name("p")
                where = row.find_elements_by_tag_name("h4")
                i = len(where)
                for sth in plist:
                    a = sth.find_elements_by_tag_name("span")
                    for _ in range(i):
                        if a[0].text in in_lib:
                            status = '書在館'
                        print('交通大學', where[_].text, status, w, end = "\n")
                        status = '不在圖書館內'
                        break
                    break
            

        elif 'nthu' in web: #清華系統
            time.sleep(5)
            where = driver.find_elements_by_link_text('全部館藏')[1].click()
            time.sleep(5)
            table = driver.find_element_by_xpath("/html/body/table[12]")
            trlist = table.find_elements_by_tag_name('tr')
            for row in trlist:
                tdlist = row.find_elements_by_tag_name('td')
                for sth in tdlist:
                    if tdlist[8].text in in_lib:
                        status = '書在館'
                    print('清華大學', tdlist[2].text, status, w, end = "\n")
                    status = '不在圖書館內'
                     break

        elif 'ncu' in web: #中央系統
            trlist = driver.find_elements_by_class_name('bibItemsEntry')
            for row in trlist:
                tdlist = row.find_elements_by_tag_name('td')
                if tdlist[3].text in in_lib:
                    status = '書在館'
                print('中央大學', tdlist[0].text, status, w, end = "\n")
                status = '不在圖書館內'

        elif 'ncl' in web: #國圖系統(館藏狀態保留原本)
            where = driver.find_element_by_link_text("書在哪裡(請點選)").click()
            time.sleep(2)
            tdlist = driver.find_elements_by_class_name('td1')
            print('國家圖書館', tdlist[1].text, tdlist[6].text, w, end = "\n")
            

        elif 'las.sinica' in web: #中研院系統
            trlist = driver.find_elements_by_class_name('bibItemsEntry')
            for row in trlist:
                tdlist = row.find_elements_by_tag_name('td')
                if tdlist[2].text in in_lib:
                    status = '書在館'
                print('中央研究院',tdlist[0].text, status, w, end = "\n")
                status = '不在圖書館內'

        elif 'ntpu' in web: #北大系統
            table = driver.find_element_by_class_name("book_location")
            trlist = table.find_elements_by_class_name('odd')
            for row in trlist:
                tdlist = row.find_elements_by_tag_name('td')
                for sth in tdlist:
                    if tdlist[2].text in in_lib:
                        status = '書在館'
                    print('台北大學', '圖書館', status, w, end = "\n")
                    status = '不在圖書館內'
                    break

        elif '203.64.5.158' in web: #台北藝術大學系統
            where = driver.find_element_by_xpath("/html/body/div/div[1]/div[2]/div/div/div[2]/div[3]/div[1]/div[3]/div/ul/li/div/div[2]/h3/a").click()
            time.sleep(2)
            table = driver.find_element_by_class_name('table.table-bordered')
            trlist = table.find_elements_by_tag_name('tr')
            for row in trlist:
                tdlist = row.find_elements_by_tag_name('td')
                for sth in tdlist:
                    if tdlist[5].text in in_lib:
                        status = '書在館'
                    print('台北藝術大學', tdlist[0].text, status, w, end = "\n")
                    status = '不在圖書館內'
                    break

        elif '140.131.94.8' in web: #護理健康大學系統
            table = driver.find_element_by_xpath("/html/body/div[4]/div[1]/form[1]/div[2]/div[1]/ul[3]/li[1]/ul[3]/li[2]/table")
            trlist = table.find_elements_by_tag_name('tr')
            for _ in trlist:
                tdlist = driver.find_elements_by_class_name('holdingslist')
                if ('到期日' not in tdlist[3].text) or ('訂購中' not in tdlist[3].text):
                    status = '書在館'
                print('台北護理健康大學','圖書館' , status, w, end = "\n")
                status = '不在圖書館內'
                break

        elif 'ntut' in web: #北科系統
            table = driver.find_element_by_class_name('order')
            trlist = table.find_elements_by_tag_name('tr')
            for row in trlist:
                tdlist = row.find_elements_by_tag_name('td')
                for sth in tdlist:
                    if tdlist[8].text in in_lib:
                        status = '書在館'
                    print('台北科技大學', tdlist[2].text, status, w, end = "\n")
                    status = '不在圖書館內'
                    break    

        elif 'ntl' in web: #國台圖系統
            where = driver.find_element_by_xpath("/html/body/table[8]/tbody/tr[1]/td[2]/a").click()
            time.sleep(2)
            tdlist = driver.find_elements_by_class_name('td1')
            if tdlist[14].text in in_lib:
                    status = '書在館'
            print('國立台灣圖書館', tdlist[7].text, status, w, end = "\n") 
            status = '不在圖書館內'

        elif 'itrilib' in web: #工研院系統
            where = driver.find_element_by_xpath("/html/body/table[8]/tbody/tr[1]/td[2]/a").click()
            time.sleep(2)
            tdlist = driver.find_elements_by_class_name('td1')
            if tdlist[12].text in in_lib:
                status = '書在館'
            print('工業研究院', tdlist[9].text, status, w, end = "\n")
            status = '不在圖書館內'

        elif 'nhri' in web: #國衛院系統
            where = driver.find_element_by_xpath('/html/body/table[8]/tbody/tr[1]/td[2]/a').click()
            time.sleep(2)
            tdlist = driver.find_elements_by_class_name('td1')
            if tdlist[13].text in in_lib:
                status = '書在館'
            print('國家衛生研究院', tdlist[6].text, status, w, end = "\n")
            status = '不在圖書館內'

        elif 'webpac.pccu' in web: #中國文化大學系統
            trlist = driver.find_elements_by_class_name('bibItemsEntry')
            for row in trlist:
                tdlist = row.find_elements_by_tag_name('td')
                if tdlist[3].text in in_lib:
                    status = '書在館'
                print('中國文化大學', tdlist[0].text, status, w, end = "\n")
                status = '不在圖書館內'

        elif 'shu' in web: #世新系統       
            where = driver.find_element_by_link_text("世新圖書館").click()
            table = driver.find_element_by_xpath("/html/body/p[2]/table[1]")
            trlist = table.find_elements_by_tag_name('tr')
            for row in trlist:
                tdlist = row.find_elements_by_tag_name('td')
                for sth in tdlist:
                    if tdlist[1].text in in_lib:
                        status = '書在館'
                    print('世新大學', '圖書館', status, w, end = "\n")
                    status = '不在圖書館內'
                    break

        elif 'uco-network' in web: #淡江系統(觀察中)
            time.sleep(5)
            try:
                back = driver.find_element_by_css_selector(".tab-header .back-button.button-with-icon.zero-margin.md-button.md-primoExplore-theme.md-ink-ripple")
            except:
                back = None
            if back != None:
                back.click()
            thelist = driver.find_elements_by_class_name("md-2-line.md-no-proxy._md")
            for row in thelist:
                plist = row.find_elements_by_tag_name("p")
                where = row.find_elements_by_tag_name("h3")
                i = len(where)
                for sth in plist:
                    a = sth.find_elements_by_tag_name("span")
                    for _ in range(i):
                        if a[0].text in in_lib:
                            status = '書在館'
                        print('淡江大學', where[_].text, status, a[2].text, w, end = "\n")
                        status = '不在圖書館內'
                        break
                    break
                break

        elif 'fju' in web: #輔仁大學系統
            trlist = driver.find_elements_by_class_name('bibItemsEntry')
            for row in trlist:
                tdlist = row.find_elements_by_tag_name('td')
                if tdlist[3].text in in_lib:
                        status = '書在館'
                print('輔仁大學', tdlist[0].text, status, w, end = "\n")
                status = '不在圖書館內'

        elif 'mcu' in web: #銘傳系統
            time.sleep(5)
            try:
                back = driver.find_element_by_css_selector(".tab-header .back-button.button-with-icon.zero-margin.md-button.md-primoExplore-theme.md-ink-ripple")
            except:
                back = None
            if back != None:
                back.click()
            thelist = driver.find_elements_by_class_name("md-2-line.md-no-proxy._md")
            for row in thelist[:-2]:
                plist = row.find_elements_by_tag_name("p")
                where = row.find_elements_by_tag_name("h3")
                i = len(where)
                for sth in plist:
                    a = sth.find_elements_by_tag_name("span")
                    for _ in range(1):
                        if a[0].text in in_lib:
                            status = '書在館'
                        print('銘傳大學', where[_].text, status, w, end = "\n")
                        status = '不在圖書館內'
                        break
                    break
                

        elif 'scu' in web: #東吳系統
            time.sleep(5)
            try:
                back = driver.find_element_by_css_selector(".tab-header .back-button.button-with-icon.zero-margin.md-button.md-primoExplore-theme.md-ink-ripple")
            except:
                back = None
            if back != None:
                back.click()
            thelist = driver.find_elements_by_class_name("md-2-line.md-no-proxy._md")
            for row in thelist[:-2]:
                plist = row.find_elements_by_tag_name("p")
                where = row.find_elements_by_tag_name("h3")
                i = len(where)
                for sth in plist:
                    a = sth.find_elements_by_tag_name("span")
                    for _ in range(1):
                        if a[0].text in in_lib:
                            status = '書在館'
                        print('東吳大學', where[_].text, status, w, end = "\n")
                        status = '不在圖書館內'
                        break
                    break
                

        elif 'cylis' in web: #中原大學系統
            trlist = driver.find_elements_by_class_name('bibItemsEntry')
            for row in trlist:
                tdlist = row.find_elements_by_tag_name('td')
                if tdlist[3].text in in_lib:
                    status = '書在館'
                print('中原大學', tdlist[0].text, status, w, end = "\n")
                status = '不在圖書館內'

        elif 'tmu' in web: #北醫系統
            table = driver.find_element_by_class_name('order')
            trlist = table.find_elements_by_tag_name('tr')
            for row in trlist:
                tdlist = row.find_elements_by_tag_name('td')
                for sth in tdlist:
                    if tdlist[5].text in in_lib:
                        status = '書在館'
                    print('台北醫學大學', tdlist[2].text, status, w, end = "\n")
                    status = '不在圖書館內'
                    break

        elif 'moc' in web: #台博館系統
            where = driver.find_element_by_link_text("書在哪裡 ? (請點我)").click()
            table = driver.find_element_by_xpath("/html/body/table[9]")
            trlist = table.find_elements_by_tag_name('tr')
            for row in trlist:
                tdlist = row.find_elements_by_tag_name('td')
                for sth in tdlist:
                    if tdlist[7].text in in_lib:
                        status = '書在館'
                    print(tdlist[2].text, tdlist[3].text, status, w, end = "\n")
                    status = '不在圖書館內'
                    break

        elif 'jinwen' in web: #景文科大系統(本來就是書在館)
            table = driver.find_element_by_class_name('order')
            trlist = table.find_elements_by_tag_name('tr')
            for row in trlist:
                tdlist = row.find_elements_by_tag_name('td')
                for sth in tdlist:
                    if tdlist[5].text in in_lib:
                        status = '書在館'
                    print('景文科技大學', tdlist[2].text, status, w, end = "\n")
                    status = '不在圖書館內'
                    break

        elif 'mmc' in web: #馬偕醫護系統
            where = driver.find_element_by_xpath("/html/body/table[8]/tbody/tr[1]/td[2]/span/a[1]").click()
            time.sleep(2)
            tdlist = driver.find_elements_by_class_name('td1')
            if tdlist[7].text in in_lib:
                status = '書在館'
            print('馬偕醫護管理專科學校', tdlist[1].text, status, w, end = "\n")
            status = '不在圖書館內'

        elif 'tfi' in web: #視聽文化中心系統
            table = driver.find_element_by_xpath("/html/body/div/div[1]/div/div/div/div[2]/div/div[2]/div[3]/div/div[4]")
            trlist = table.find_elements_by_tag_name('tr')
            for row in trlist:
                tdlist = row.find_elements_by_tag_name('td')
                for sth in tdlist:
                    if tdlist[4].text in in_lib:
                        status = '書在館'
                    print('國家視聽文化中心', tdlist[1].text, status, w, end = "\n")
                    status = '不在圖書館內'
                    break   


                    
driver.close()

986729193X


<ipython-input-12-0ec7cf7ee831>:39: DeprecationWarning: use driver.switch_to.alert instead
  driver.switch_to_alert().accept() # 點選彈出裡面的確定按鈕


台灣大學 總圖書館 書在館 https://ntu.primo.exlibrisgroup.com/discovery/fulldisplay?context=L&vid=886NTU_INST:886NTU_INST&lang=zh-tw&docid=alma991032746609704786
交通大學 交大圖書總館 書在館 http://ustcate.lib.nctu.edu.tw/NCTU:NCTU_ALEPH001004060
清華大學 人社分館 不在圖書館內 http://webpac.lib.nthu.edu.tw/F?func=direct&l_base=TOP01&format=999&doc_number=000273599
清華大學 總圖 不在圖書館內 http://webpac.lib.nthu.edu.tw/F?func=direct&l_base=TOP01&format=999&doc_number=000273599
清華大學 總圖 不在圖書館內 http://webpac.lib.nthu.edu.tw/F?func=direct&l_base=TOP01&format=999&doc_number=000273599
清華大學 南大分館 不在圖書館內 http://webpac.lib.nthu.edu.tw/F?func=direct&l_base=TOP01&format=999&doc_number=000273599
政治大學 綜圖 書在館 https://nccu.primo.exlibrisgroup.com/discovery/fulldisplay?context=L&vid=886NCCU_INST:886NCCU_INST&lang=zh-tw&docid=alma991010195759705721
政治大學 達賢館 書在館 https://nccu.primo.exlibrisgroup.com/discovery/fulldisplay?context=L&vid=886NCCU_INST:886NCCU_INST&lang=zh-tw&docid=alma991010195759705721
國立台灣圖書館 圖書館 不在圖書館內 http://cis.ntl.edu.tw/F?func=direct&

NameError: name 'ime' is not defined

In [9]:
#可以爬出各大學和超連結版+輸出list+館藏狀態 稍微改過狀態統一的版本

from selenium import webdriver # 先下載 webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
import time

driver = webdriver.Chrome("C:\\Users\\88698\\OneDrive\\文件\\chromedriver") #記得改位置
driver.get("https://metacat.ntu.edu.tw/") # 更改網址以前往不同網頁

#改成用ISBN查
ISBN = input()
element = driver.find_element_by_name('simpleSearchText')
element.send_keys(ISBN)
select = Select(driver.find_element_by_id('simpleType'))
select.select_by_value("ISBN")

# 把不要的勾掉
choose_btn = driver.find_element_by_link_text("機構單位篩選").click()
time.sleep(1)

btn_mid = driver.find_element_by_id('library1').click()
btn_south = driver.find_element_by_id('library2').click()
btn_east = driver.find_element_by_id('library3').click()

no_hsc = driver.find_element_by_id('hsc').click() # 新生醫專
no_tust = driver.find_element_by_id('tust').click() # 大華科大
no_must = driver.find_element_by_id('must').click() # 明新科大
no_taitheo = driver.find_element_by_id('taitheo').click() # 台灣神學研究學院
no_dila = driver.find_element_by_id('dila').click() # 法鼓文理學院
no_yzu = driver.find_element_by_id('yzu').click() # 元智大學
no_niu = driver.find_element_by_id('niu').click() # 宜蘭大學
no_lhu = driver.find_element_by_id('lhu').click() # 龍華科大
no_oit = driver.find_element_by_id('oit').click() # 亞東技術學院
no_ntuvvAlma = driver.find_element_by_id('ntuvvAlma').click() # 原住民圖資中心

save_opt = driver.find_element_by_id("saveOptions").click() # 儲存已選選項
driver.switch_to_alert().accept() # 點選彈出裡面的確定按鈕
close = driver.find_element_by_class_name("close").click() # 按叉叉

search_gogogo = driver.find_element_by_id('simpleSearchButton').click()
time.sleep(70)

# 有"顯示更多"就按下去
more = driver.find_elements_by_name('collapseLink')
for i in range(len(more)):
    more[i].click()
    time.sleep(1)

# 每頁顯示 100 項搜尋結果
try:
    show = Select(driver.find_element_by_name("resultTable_length"))
    show.select_by_value("100")
    
    #爬 Metacat
    name = []
    books = driver.find_elements_by_class_name('institution-list')
    for i in range(len(books)):
        name.append(books[i].text)

    URL = []
    www = driver.find_elements_by_class_name('institution-list')
    for i in range(len(www)):
        website = www[i].get_attribute('href')
        if website != None:
            URL.append(website)
        else:
            www[i].click()
            www2 = driver.find_elements_by_class_name('institution-list')
            URL.append(www2[-1].get_attribute('href'))
            ActionChains(driver).move_by_offset(150, 200).click().perform()
            ActionChains(driver).move_by_offset(-150, -200).perform()

    #爬各大學網站
    for w in URL:
        web = str(w)
        
        time.sleep(8)
        status = '不在圖書館內'
        in_lib = ['可外借', '在架上', '在架', '仍在館內', '有可用館藏', '書在館', '目前可獲得', '可獲得', '在書架上', '限館內閱覽', '在館內']

        if 'ntu.primo' in web: #台大系統
            driver.get(web)
            time.sleep(5)
            try:
                back = driver.find_element_by_css_selector(".tab-header .back-button.button-with-icon.zero-margin.md-button.md-primoExplore-theme.md-ink-ripple")
            except:
                back = None
            if back != None:
                back.click()
            thelist = driver.find_elements_by_class_name("md-2-line.md-no-proxy._md")
            for row in thelist:
                plist = row.find_elements_by_tag_name("p")
                where = row.find_elements_by_tag_name("h3")
                i = len(where)
                for sth in plist:
                    a = sth.find_elements_by_tag_name("span")
                    for _ in range(i):
                        if a[0].text in in_lib:
                            status = '書在館'
                        print('台灣大學', where[_].text, status, w, end = "\n")
                        print(a[0].text)
                        status = '不在圖書館內'
                        break
                    break

except:
    output_nothing = True

9570825685


<ipython-input-9-a3116d30b62e>:39: DeprecationWarning: use driver.switch_to.alert instead
  driver.switch_to_alert().accept() # 點選彈出裡面的確定按鈕
